# Audio Sentiment Analysis with Pydantic AI and Gemini

This notebook demonstrates how to use Pydantic AI with Gemini to analyze emotions and sentiment in audio.

## Import Libraries and Load Environment

Before starting, make sure you have placed your Google Gemini credentials in the `.env` file:

```bash
cp env.example .env
```
then edit `.env` and modify GEMINI_API_KEY with your key.

In [ ]:
# Udacity workspace only. Comment out if running locally. 
# Fix for huggingface datasets when dealing with
# read-only file-systems
import filelock
import contextlib

# Create a proper no-op FileLock class
class NoOpFileLock:
    def __init__(self, lock_file, *args, **kwargs):
        pass
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        pass

# Replace FileLock with a no-op context manager
filelock.FileLock = NoOpFileLock

In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent, BinaryContent
from datasets import load_dataset
import numpy as np
from IPython.display import Audio, display
import io
import soundfile as sf

# Load environment variables
assert load_dotenv(), "Please prepare a .env file with your GEMINI_API_KEY"
assert os.getenv("GEMINI_API_KEY"), "GEMINI_API_KEY not found in .env file"

# This is needed to use asyncio within jupyter
import nest_asyncio

nest_asyncio.apply()

## Define Response Schema

We'll use Pydantic models to structure our sentiment analysis output. This allows us to make sure that the output from our system is always consistent and precise, so we can leverage it downstream to make decisions. For example, if our AI was part of a audio analysis system in a call center, we could decide to send an alert if we detect anger or other negative emotions. Having that in a structure that is easy to parse - instead of in a arbitrary text string - would make this very easy.

In [2]:
from typing import List
from typing_extensions import Literal


# Pydantic AI expects a pydantic model for structured outputs
class EmotionAnalysis(BaseModel):
    """Structured output for audio emotion analysis."""

    # We allow the model to choose more than one answer
    primary_emotion: List[
        Literal[
            "anger",
            "disgust",
            "fear",
            "happiness",
            "surprise_or_disappointment",
            "sadness",
            "neutral",
        ]
    ] = Field(description="The dominant emotion detected in the audio")
    # NOTE: do not do this. LLMs are not good at estimating their own
    # confidence this way. There are ways to do it better, e.g. via
    # log-probs or majority voting, but we won't go into this here
    # confidence: float = Field(
    #    ge=0, le=1, description="Confidence score for the primary emotion (0-1)"
    # )
    sentiment_polarity: str = Field(
        description="Overall sentiment: positive, negative, or neutral"
    )
    intensity: str = Field(description="Emotion intensity: low, medium, or high")
    description: str = Field(
        description="Brief description of the emotional tone and characteristics"
    )

## Create Pydantic AI Agent

Set up the agent with Gemini for audio sentiment analysis.

In [ ]:
from pydantic_ai.models.google import GoogleModelSettings


# Remove thinking to avoid long delays and timeouts
settings = GoogleModelSettings(
    google_thinking_config={"thinking_budget": 0},
)

emotion_agent = Agent(
    # This can work with any Gemini model,
    # we pick the cheapest and faster for now
    model="gemini-2.5-flash-lite",
    output_type=EmotionAnalysis,
    instructions="""
    You are an expert audio sentiment analyzer. Listen to the provided audio and analyze:
    1. The primary emotions expressed by the speaker
    2. The overall sentiment polarity (positive/negative/neutral)
    3. The intensity of the emotional expression
    4. A brief description of the emotional characteristics and a rationale for 
       your choices

    Be precise and analytical in your assessment.
    """,
    model_settings=settings,
)

## Load Dataset

Load a sample from an academic audio dataset, widely used in academic publications on sentiment analysis in audio:

In [5]:
dataset = load_dataset("ajyy/MELD_audio", split="train", trust_remote_code=True)

# Print the emotions available in the dataset
# NOTE: it does not matter that they don't correspond 1-1 to what we 
# defined above, as we won't be using these labels. They are not very
# accurate.
emotions = set(dataset["emotion"])

print(f"Dataset loaded with {len(dataset)} samples")
print(f"Available emotions: {emotions}")
print(f"Sample structure: {dataset.features}")

# Show first few samples
for i in range(3):
    display(
        Audio(dataset[i]["audio"]["array"], rate=dataset[i]["audio"]["sampling_rate"])
    )
    print(f"Sample {i}: {dataset[i]['text']} - {dataset[i]['emotion']})")

Dataset loaded with 9988 samples
Available emotions: {'surprise', 'sadness', 'joy', 'disgust', 'fear', 'anger', 'neutral'}
Sample structure: {'text': Value(dtype='string', id=None), 'path': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'emotion': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}


Sample 0: also I was the point person on my companys transition from the KL-5 to GR-6 system. - neutral)


Sample 1: You mustve had your hands full. - neutral)


Sample 2: No dont I beg of you! - fear)


## Helper Function for Audio Processing

Let's define some helper functions to format our audio:

In [6]:
def audio_to_bytes(audio_array, sample_rate):
    """Convert audio array to bytes for Pydantic AI."""
    buffer = io.BytesIO()
    sf.write(buffer, audio_array, sample_rate, format='WAV')
    buffer.seek(0)
    return buffer.getvalue()

def format_audio_for_gemini(sample_idx):
    """Analyze a single audio sample from the dataset."""
    sample = dataset[sample_idx]
    
    # Convert audio to bytes
    audio_bytes = audio_to_bytes(
        sample['audio']['array'], 
        sample['audio']['sampling_rate']
    )
    
    # Create binary content for Pydantic AI
    audio_content = BinaryContent(
        data=audio_bytes,
        media_type='audio/wav'
    )
    
    return sample, audio_content

## Single Audio Analysis

Analyze one audio sample to see the structured output.

In [9]:
# Select a sample to analyze
import textwrap


sample_idx = 10
sample, audio_content = format_audio_for_gemini(sample_idx)

print(f"Analyzing sample: '{sample['text']}' - Ground truth: {sample['emotion']}")

# Display the audio
display(Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))

# Run sentiment analysis
result = emotion_agent.run_sync(
    ["Analyze the emotion and sentiment in this audio clip.", audio_content]
)

print("\nAnalysis Results:")
print(f"Primary Emotion: {result.output.primary_emotion}")
print(f"Sentiment: {result.output.sentiment_polarity}")
print(f"Intensity: {result.output.intensity}")
print(f"Description: {textwrap.fill(result.output.description)}")
print(f"\nGround Truth: {sample['emotion']}")

Analyzing sample: 'I see.' - Ground truth: neutral



Analysis Results:
Primary Emotion: ['neutral']
Sentiment: neutral
Intensity: low
Description: The speaker is calmly stating facts or observations with no
discernible emotional coloring.

Ground Truth: neutral


## Batch Analysis

Analyze multiple samples across different emotions.

In [8]:
# Select samples from the dataset across
# different emotions (as defined in the dataset)
emotion_samples = {}
emotions_to_test = {"fear", "joy", "sadness", "anger", "surprise", "disgust", "neutral"}

# Find one sample for each emotion
for emotion in emotions_to_test:
    for idx, item in enumerate(dataset):
        if item["emotion"] == emotion and emotion not in emotion_samples:
            emotion_samples[emotion] = idx
            break

print("Testing samples from different emotions:")
results = []

for emotion, idx in emotion_samples.items():
    sample, audio_content = format_audio_for_gemini(idx)

    display(Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))
    print(f"Text: '{sample['text']}'")

    # Run analysis
    result = emotion_agent.run_sync(
        ["Analyze the emotion and sentiment in this audio clip.", audio_content]
    )

    print(result.output.model_dump_json(indent=2))
    print()

Testing samples from different emotions:


Text: 'Would you look at this dump? He hated us. This is his final revenge!'
{
  "primary_emotion": [
    "anger",
    "disgust"
  ],
  "sentiment_polarity": "negative",
  "intensity": "high",
  "description": "The speaker expresses strong negative emotions, likely anger and disgust, characterized by a raised voice and sharp, accusatory tone. The phrases \"look at this dump\" and \"He hated us\" convey contempt and resentment, while the overall delivery suggests a high level of emotional intensity. The sentiment is clearly negative due to the hostile and demeaning language used."
}



Text: 'No dont I beg of you!'
{
  "primary_emotion": [
    "fear"
  ],
  "sentiment_polarity": "negative",
  "intensity": "high",
  "description": "The speaker expresses a high intensity of fear and desperation, indicated by the plea \"No, don't. I beg of you.\""
}



Text: 'Really?!'
{
  "primary_emotion": [
    "surprise_or_disappointment"
  ],
  "sentiment_polarity": "neutral",
  "intensity": "medium",
  "description": "The speaker expresses surprise or disappointment with a neutral overall sentiment. The initial utterance \"Really, absolutely\" conveys a sense of disbelief or unexpectedness, followed by a sigh that reinforces this feeling. The intensity is medium as the emotion is clearly present but not overwhelming."
}



Text: 'Actually, I think this apartment sullies the good name of crap'
{
  "primary_emotion": [
    "disgust"
  ],
  "sentiment_polarity": "negative",
  "intensity": "high",
  "description": "The speaker expresses strong disgust and disappointment with the apartment, using a derogatory term to describe it. The tone is clearly negative and the intensity is high due to the forceful language used."
}



Text: 'If it's not you, this is a horrible story.'
{
  "primary_emotion": [
    "sadness"
  ],
  "sentiment_polarity": "negative",
  "intensity": "high",
  "description": "The speaker expresses a strong sense of sadness and disappointment, possibly despair, conveyed through a somber tone and a dramatic statement about a \"horrible story.\""
}



Text: 'Yeah!'
{
  "primary_emotion": [
    "happiness"
  ],
  "sentiment_polarity": "positive",
  "intensity": "low",
  "description": "The speaker sounds pleased and content, using a mild and positive tone."
}



Text: 'also I was the point person on my companys transition from the KL-5 to GR-6 system.'
{
  "primary_emotion": [
    "neutral"
  ],
  "sentiment_polarity": "neutral",
  "intensity": "low",
  "description": "The speaker is calmly recounting a past event, stating facts about their role in a company transition. There is no discernible emotional charge or sentiment expressed in the audio."
}



As we can see, these are all reasonable choices given the inputs!